# Lab 6: Memory - Agentes que Lembram

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_before.png" style="width:500px; border:1px solid #ccc; border-radius:6px;">
</div>

## 🎯 O que você vai aprender neste Lab

**Memory** (memória) permite que agentes persistam mensagens e estado entre invocações. Sem memória, cada conversa começa do zero!

### Por que Memory é importante?

1. **Contexto conversacional**: O agente lembra o que foi dito antes
2. **Continuidade**: Perguntas de follow-up funcionam naturalmente
3. **Eficiência**: Não precisa repetir informações já fornecidas
4. **UX natural**: Conversas fluem como com humanos

### Tipos de Memória:

| Tipo | Duração | Uso | Implementação |
|------|---------|-----|---------------|
| **Curto prazo** | Uma sessão | Conversa atual | `InMemorySaver` (checkpointer) |
| **Longo prazo** | Persistente | Múltiplas sessões | Store (banco de dados) |

### O Problema Sem Memória:

```
Usuário: "Meu nome é Douglas"
Agente: "Olá Douglas!"

Usuário: "Qual é meu nome?"
Agente: "Desculpe, não sei seu nome."  ❌
```

### Com Memória:

```
Usuário: "Meu nome é Douglas"
Agente: "Olá Douglas!"

Usuário: "Qual é meu nome?"
Agente: "Seu nome é Douglas!"  ✅
```

## ⚙️ Setup - Configuração do Ambiente

Instalação das bibliotecas, incluindo `langgraph` para checkpointing.

In [ ]:
%pip install -U mlflow>=3 langchain>=1 langchain-community langgraph databricks-langchain --quiet
dbutils.library.restartPython()

In [ ]:
import mlflow
mlflow.langchain.autolog()

In [ ]:
# No Databricks, usamos spark.sql() diretamente
# Verifique as tabelas disponíveis no catálogo NASA GCN
%sql
SHOW TABLES IN nasa_gcn.silver

## 🔧 Configuração do Agente SQL (Sem Memória)

Primeiro, vamos criar um agente SQL sem memória para demonstrar o problema.

In [ ]:
# RuntimeContext não é necessário quando usamos spark diretamente

In [ ]:
from langchain_core.tools import tool

@tool
def execute_sql(query: str) -> str:
    """Execute a SQL command on Databricks and return results."""
    try:
        result = spark.sql(query).limit(5).toPandas()
        return result.to_markdown()
    except Exception as e:
        return f"Error: {e}"

In [ ]:
SYSTEM_PROMPT = """Você é um analista cuidadoso de Databricks SQL especializado em dados da NASA GCN.

Regras:
- Pense passo a passo.
- Quando precisar de dados, chame a ferramenta `execute_sql` com UMA query SELECT.
- Apenas leitura; sem INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limite a 5 linhas de saída, a menos que o usuário peça mais.
- Se a ferramenta retornar 'Error:', revise o SQL e tente novamente.
- Prefira listas explícitas de colunas; evite SELECT *.
- Use o catálogo nasa_gcn e schema silver (nasa_gcn.silver.tabela).
"""

In [ ]:
from langchain.agents import create_agent
from databricks_langchain import ChatDatabricks

# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.1,
)

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
)

## ❌ O Problema: Queries Repetidas Sem Contexto

Vamos fazer duas perguntas relacionadas ao agente **sem memória**.

### Primeira pergunta:

In [ ]:
question = "Quantos eventos existem no catálogo nasa_gcn schema silver?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

### Segunda pergunta (follow-up)

Agora perguntamos sobre "os tipos mais frequentes" - mas o agente **não lembra** da pergunta anterior e fica confuso!

## ✅ A Solução: Adicionando Memória de Curto Prazo

Vamos resolver isso usando `InMemorySaver` como **checkpointer**.

### Como funciona:

1. **Checkpointer**: Salva o estado do agente após cada invocação
2. **thread_id**: Identifica a "conversa" - mensagens no mesmo thread são lembradas
3. **Persistência**: O histórico é mantido enquanto o checkpointer existir

```python
checkpointer = InMemorySaver()  # Armazena em memória (volátil)

agent = create_agent(
    ...,
    checkpointer=checkpointer
)

# Todas as invocações com thread_id="1" compartilham histórico
agent.stream(..., {"configurable": {"thread_id": "1"}})
```

### Importando InMemorySaver

In [0]:
from langgraph.checkpoint.memory import InMemorySaver

### Criando o agente COM memória

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent_with_memory = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
)

### Testando COM memória - Primeira pergunta

Observe que agora passamos `{"configurable": {"thread_id": "1"}}` para identificar a conversa.

In [ ]:
question = "Liste as tabelas disponíveis no catálogo nasa_gcn schema silver"
steps = []

for step in agent_with_memory.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

### Segunda pergunta (follow-up) - AGORA FUNCIONA!

O agente lembra da conversa anterior e sabe do que estamos falando. Observe que ele não precisa descobrir as tabelas novamente!

In [ ]:
question = "Qual delas tem mais registros?"
steps = []

for step in agent_with_memory.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

## 🎮 Sua Vez - Teste a Memória!

Agora que o agente tem memória, teste perguntas de follow-up! Experimente:
- Referências a respostas anteriores ("desses, qual...")
- Perguntas sobre contexto ("sobre o que falamos?")
- Comparações ("compare com a anterior")

In [ ]:
question = "Sua pergunta aqui?"
steps = []

for step in agent_with_memory.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

## 📚 Resumo e Próximos Passos

### O que aprendemos:

| Conceito | Descrição |
|----------|-----------|
| **Checkpointer** | Componente que salva estado do agente |
| **InMemorySaver** | Checkpointer que armazena em memória (volátil) |
| **thread_id** | Identificador único da conversa |
| **Memória de curto prazo** | Persiste durante a sessão |

### Thread IDs:

```python
# Mesma conversa
{"configurable": {"thread_id": "1"}}

# Nova conversa (não lembra da anterior)
{"configurable": {"thread_id": "2"}}
```

### Para memória persistente (longo prazo):
- Use um checkpointer com banco de dados (PostgreSQL, SQLite, etc.)
- Ou use o `Store` do LangGraph para memória semântica

### Próximo Lab:
No **Lab 7 - Structured Output**, você aprenderá a fazer o agente retornar dados em formatos específicos (JSON, Pydantic models)!